In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import accuracy_score

In [2]:
path="../dataset/TrafficLabelling /Tuesday-WorkingHours.pcap_ISCX.csv"
t1=time.time()
df_tuesday=pd.read_csv(path)
t2=time.time()
print("time taken to read tuesday csv :: ", t2-t1, " seconds ")

/home/ebryx/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


time taken to read tuesday csv ::  5.00166654586792  seconds 


In [3]:
df_tuesday_dropped=df_tuesday.dropna( axis=0, how='any')

In [4]:
df_tuesday_dropped.replace(',,', np.nan, inplace=True)

/home/ebryx/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [5]:
df_tuesday_dup_rem = df_tuesday_dropped.loc[:,~df_tuesday_dropped.columns.duplicated()]
len(df_tuesday_dup_rem.columns)

85

In [6]:
df_tuesday_dropped.drop(columns=[' Fwd Header Length.1'], axis=1, inplace=True)

/home/ebryx/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
len(df_tuesday_dropped.columns)


84

In [8]:
from sklearn import preprocessing      # import required package for labelencoding
le=preprocessing.LabelEncoder()        # create a label encoder (category) object
le.fit(df_tuesday_dropped['Flow ID'])

LabelEncoder()

In [9]:
# labels
le.classes_
# Apply fitted encoder to the pandas column
le.transform(df_tuesday_dropped["Flow ID"])

array([152266, 152267, 152268, ..., 157091,  79335, 174989])

In [10]:
le_s_ip=preprocessing.LabelEncoder()
le_s_ip.fit(df_tuesday_dropped[' Source IP'])
le_s_ip.transform(df_tuesday_dropped[' Source IP'])

array([2145, 2145, 2145, ..., 2146, 5380, 2149])

In [11]:
le_d_ip=preprocessing.LabelEncoder()
le_d_ip.fit(df_tuesday_dropped[' Destination IP'])
le_d_ip.transform(df_tuesday_dropped[' Destination IP'])

array([2558, 2558, 2558, ..., 2558, 2556, 2558])

In [12]:
le_time_stamp=preprocessing.LabelEncoder()
le_time_stamp.fit(df_tuesday_dropped[' Timestamp'])
le_time_stamp.transform(df_tuesday_dropped[' Timestamp'])

array([422, 422, 422, ..., 439, 168,  38])

In [13]:
group=df_tuesday_dropped.groupby(' Label')
group.first()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
Label,,,,,,,,,,,,,,,,,,,,,
BENIGN,192.168.10.3-192.168.10.5-88-49182-6,192.168.10.5,49182,192.168.10.3,88,6,4/7/2017 8:54,640,7,4,...,2,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FTP-Patator,172.16.0.1-192.168.10.50-45422-80-6,172.16.0.1,45422,192.168.10.50,80,6,4/7/2017 10:30,5216127,3,1,...,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SSH-Patator,172.16.0.1-192.168.10.50-52358-22-6,172.16.0.1,52358,192.168.10.50,22,6,4/7/2017 3:11,404,2,0,...,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df_tuesday_dropped[[' Flow Packets/s', 'Flow Bytes/s']].replace('Infinity', -1, inplace=True)
df_tuesday_dropped[[' Flow Packets/s', 'Flow Bytes/s']].replace('NaN', 0, inplace=True)

In [15]:
df=df_tuesday_dropped

In [16]:
features=[" Fwd Packet Length Max"," Flow IAT Std"," Fwd Packet Length Std" ,"Fwd IAT Total",' Flow Packets/s',' Flow Duration', " Fwd Packet Length Mean", " Total Length of Bwd Packets", "Flow Bytes/s", "Total Length of Fwd Packets", " Flow IAT Mean", " Bwd Packet Length Mean",  " Flow IAT Max", " Bwd Packet Length Std", ' Total Fwd Packets', ' Total Backward Packets'," Label"]
data_frame_for_patator=df[features].copy()
data_frame_for_patator.head()

,Fwd Packet Length Max,Flow IAT Std,Fwd Packet Length Std,Fwd IAT Total,Flow Packets/s,Flow Duration,Fwd Packet Length Mean,Total Length of Bwd Packets,Flow Bytes/s,Total Length of Fwd Packets,Flow IAT Mean,Bwd Packet Length Mean,Flow IAT Max,Bwd Packet Length Std,Total Fwd Packets,Total Backward Packets,Label
0,220,135.557286,107.349008,640.0,17187.5,640,62.857143,358.0,1246875,440,64.000000,89.5,445.0,103.345698,7,4,BENIGN
1,300,192.795228,132.287566,900.0,14444.44444,900,66.666667,2944.0,3937777.778,600,75.000000,736.0,684.0,849.859596,9,4,BENIGN
2,1388,236.433336,677.274651,1205.0,9128.630705,1205,396.571429,2830.0,4652282.158,2776,120.500000,707.5,777.0,816.950631,7,4,BENIGN
3,226,90.767652,110.276708,511.0,21526.41879,511,64.571429,370.0,1608610.568,452,51.100000,92.5,299.0,106.809800,7,4,BENIGN
4,306,148.698266,134.933317,773.0,16817.59379,773,68.000000,2944.0,4600258.732,612,64.416667,736.0,531.0,849.859596,9,4,BENIGN


In [17]:
df=data_frame_for_patator

In [18]:
df.head()

,Fwd Packet Length Max,Flow IAT Std,Fwd Packet Length Std,Fwd IAT Total,Flow Packets/s,Flow Duration,Fwd Packet Length Mean,Total Length of Bwd Packets,Flow Bytes/s,Total Length of Fwd Packets,Flow IAT Mean,Bwd Packet Length Mean,Flow IAT Max,Bwd Packet Length Std,Total Fwd Packets,Total Backward Packets,Label
0,220,135.557286,107.349008,640.0,17187.5,640,62.857143,358.0,1246875,440,64.000000,89.5,445.0,103.345698,7,4,BENIGN
1,300,192.795228,132.287566,900.0,14444.44444,900,66.666667,2944.0,3937777.778,600,75.000000,736.0,684.0,849.859596,9,4,BENIGN
2,1388,236.433336,677.274651,1205.0,9128.630705,1205,396.571429,2830.0,4652282.158,2776,120.500000,707.5,777.0,816.950631,7,4,BENIGN
3,226,90.767652,110.276708,511.0,21526.41879,511,64.571429,370.0,1608610.568,452,51.100000,92.5,299.0,106.809800,7,4,BENIGN
4,306,148.698266,134.933317,773.0,16817.59379,773,68.000000,2944.0,4600258.732,612,64.416667,736.0,531.0,849.859596,9,4,BENIGN


###  save df for use later

In [20]:
df.to_csv("../dataset/Cleaned/multiclass-tuesday.csv")

In [13]:
df=pd.read_csv("../dataset/Cleaned/multiclass-tuesday.csv")

/home/ebryx/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
X=df[df.columns[0:-1]]
y=df[df.columns[-1]]
X.dtypes

Unnamed: 0                        int64
 Fwd Packet Length Max            int64
 Flow IAT Std                   float64
 Fwd Packet Length Std          float64
Fwd IAT Total                   float64
 Flow Packets/s                  object
 Flow Duration                    int64
 Fwd Packet Length Mean         float64
 Total Length of Bwd Packets    float64
Flow Bytes/s                     object
Total Length of Fwd Packets       int64
 Flow IAT Mean                  float64
 Bwd Packet Length Mean         float64
 Flow IAT Max                   float64
 Bwd Packet Length Std          float64
 Total Fwd Packets                int64
 Total Backward Packets           int64
dtype: object

In [15]:
X.replace("Infinity", 0, inplace=True)
X['Flow Bytes/s'].replace("Infinity", 0,inplace=True)
X[" Flow Packets/s"].replace("Infinity", 0, inplace=True)

/home/ebryx/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/home/ebryx/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [5]:
X["Flow Bytes/s"]=X["Flow Bytes/s"].astype("float")
X[' Flow Packets/s']=X[" Flow Packets/s"].astype("float")
X.dtypes

/home/ebryx/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ebryx/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Unnamed: 0                        int64
 Fwd Packet Length Max            int64
 Flow IAT Std                   float64
 Fwd Packet Length Std          float64
Fwd IAT Total                   float64
 Flow Packets/s                 float64
 Flow Duration                    int64
 Fwd Packet Length Mean         float64
 Total Length of Bwd Packets    float64
Flow Bytes/s                    float64
Total Length of Fwd Packets       int64
 Flow IAT Mean                  float64
 Bwd Packet Length Mean         float64
 Flow IAT Max                   float64
 Bwd Packet Length Std          float64
 Total Fwd Packets                int64
 Total Backward Packets           int64
dtype: object

In [16]:
from scipy import stats

In [21]:
t1=time.time()

cols = list(X.columns)
for col in cols:
    print(X[col])
#     X[col] = stats.zscore(X[col])
    
t2=time.time()
print("time ::: ", t2-t1 , " seconds to find z score of 13 features")

0        -1.732072
1        -1.732064
2        -1.732056
3        -1.732049
4        -1.732041
            ...   
445703    1.732064
445704    1.732072
445705    1.732079
445706    1.732087
445707    1.732095
Name: Unnamed: 0, Length: 445708, dtype: float64
0         0.098563
1         0.267860
2         2.570299
3         0.111260
4         0.280557
            ...   
445703   -0.273890
445704   -0.367004
445705   -0.273890
445706   -0.367004
445707   -0.259077
Name:  Fwd Packet Length Max, Length: 445708, dtype: float64
0        -0.252573
1        -0.252563
2        -0.252555
3        -0.252580
4        -0.252570
            ...   
445703   -0.252582
445704   -0.252596
445705   -0.252580
445706   -0.252596
445707   -0.252583
Name:  Flow IAT Std, Length: 445708, dtype: float64
0         0.391301
1         0.561222
2         4.274543
3         0.411249
4         0.579249
            ...   
445703   -0.340132
445704   -0.340132
445705   -0.340132
445706   -0.340132
445707   -0.340132
Na

In [ ]:
from sklearn.model_selection import train_test_split     # import module for train test split

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=10)

In [ ]:
X_test.head(1000)

### KNN

In [10]:
from sklearn.neighbors import KNeighborsClassifier

In [15]:
t1=time.time()

for i in range(1,21):
    knn=KNeighborsClassifier(n_neighbors=i)
    model_knn=knn.fit(X_train,y_train)
    yhat=model_knn.predict(X_test)
    print("for " , i,  " as K, accuracy is : ", accuracy_score(y_test, yhat))
t2=time.time()
print(" time for ", i ," k's :: ", (t2-t1)/60 , " minutes")

for  1  as K, accuracy is :  0.9905656144129591
for  2  as K, accuracy is :  0.9933140382760091
for  3  as K, accuracy is :  0.9927867896165669
for  4  as K, accuracy is :  0.9936281438603576
for  5  as K, accuracy is :  0.9933813466155124
for  6  as K, accuracy is :  0.9937964147091158
for  7  as K, accuracy is :  0.9937178883130287
for  8  as K, accuracy is :  0.9937739785959481
for  9  as K, accuracy is :  0.9936954521998609
for  10  as K, accuracy is :  0.9937178883130287
for  11  as K, accuracy is :  0.9937066702564448
for  12  as K, accuracy is :  0.9938188508222835
for  13  as K, accuracy is :  0.9937291063696125
for  14  as K, accuracy is :  0.9937739785959481
for  15  as K, accuracy is :  0.9937291063696125
for  16  as K, accuracy is :  0.9937739785959481
for  17  as K, accuracy is :  0.993785196652532
for  18  as K, accuracy is :  0.9938525049920351
for  19  as K, accuracy is :  0.9938525049920351
for  20  as K, accuracy is :  0.9938861591617868
 time for  20  k's ::  68.8117

In [11]:
t1=time.time()

for i in range(25,30):
    knn=KNeighborsClassifier(n_neighbors=i)
    model_knn=knn.fit(X_train,y_train)
    yhat=model_knn.predict(X_test)
    print("for " , i+25,  " as K, accuracy is : ", accuracy_score(y_test, yhat))
t2=time.time()
print(" time for ", 5," k's :: ", (t2-t1)/60 , " minutes")

for  50  as K, accuracy is :  0.9937515424827803
for  51  as K, accuracy is :  0.9937291063696125
for  52  as K, accuracy is :  0.9936393619169415
for  53  as K, accuracy is :  0.993684234143277
for  54  as K, accuracy is :  0.993594489690606
 time for  5  k's ::  16.510736711819966  minutes


### Naive Bayes

In [10]:
from sklearn.naive_bayes import GaussianNB

In [11]:
t1=time.time()

gnb = GaussianNB()
model_gnb = gnb.fit(X_train, y_train)
yhat=model_gnb.predict(X_test)

print("Accuracy :: ", accuracy_score(y_test,yhat))

t2=time.time()
print(" time for Naive Baye's :: ", (t2-t1) , " seconds")

Accuracy ::  0.4973076664198694
 time for Naive Baye's ::  0.8724572658538818  seconds


In [12]:
from sklearn.naive_bayes import BernoulliNB

In [13]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
yhat=clf.predict(X_test)
print("Accuracy ::: ", accuracy_score(y_test,yhat))

Accuracy :::  0.981692131655112
